# Airbnb Berlin Price Prediction 
---


### *Table of Contents*
- [Exploratory Data Analysis](#eda)
- [Preprocessing](#preprocessing)
- [Modeling](#modeling)

In [3]:
import pandas as pd
import numpy as np

#
import matplotlib.pyplot as plt

#
from sklearn.model_selection import train_test_split

import os

### Import data

In [4]:
data_dir = "data"

In [5]:
trainset = pd.read_csv('data/train_airbnb_berlin.csv')
X_train = trainset.drop("Price", axis=1)
y_train = trainset["Price"]

In [ ]:
import pandas as pd